In [1]:
# importing required packages
import numpy as np
import cv2 

In [2]:
# load our serialized model from disk

print('[INFO] loading model...')
net= cv2.dnn.readNetFromCaffe('deploy.prototxt','res10_300x300_ssd_iter_140000.caffemodel')

# deply.prototxt defines the architecture of model
# .caffemodel have the actual weights

image= cv2.imread('data/img.jpg')
image= cv2.resize(image, (300,300))
(h,w,d )= image.shape
print("Shape of image:", image.shape)

# calculating mean of RGB layers
b_mean= np.mean(image[:,:,0])
g_mean= np.mean(image[:,:,1])
r_mean= np.mean(image[:,:,2])
sub_mean= (b_mean, g_mean, r_mean)

blob = cv2.dnn.blobFromImage(image, 1.0, (300,300), sub_mean)
print("Shape of blob:", blob.shape)

[INFO] loading model...
Shape of image: (300, 300, 3)
Shape of blob: (1, 3, 300, 300)


In [3]:
help(cv2.dnn.blobFromImage)

# inshort blobFromImage takes care of all the preprocessing 
# creates a 4d array

Help on built-in function blobFromImage:

blobFromImage(...)
    blobFromImage(image[, scalefactor[, size[, mean[, swapRB[, crop[, ddepth]]]]]]) -> retval
    .   @brief Creates 4-dimensional blob from image. Optionally resizes and crops @p image from center,
    .        *  subtract @p mean values, scales values by @p scalefactor, swap Blue and Red channels.
    .        *  @param image input image (with 1-, 3- or 4-channels).
    .        *  @param size spatial size for output image
    .        *  @param mean scalar with mean values which are subtracted from channels. Values are intended
    .        *  to be in (mean-R, mean-G, mean-B) order if @p image has BGR ordering and @p swapRB is true.
    .        *  @param scalefactor multiplier for @p image values.
    .        *  @param swapRB flag which indicates that swap first and last channels
    .        *  in 3-channel image is necessary.
    .        *  @param crop flag which indicates whether image will be cropped after resize o

In [4]:
# pass the blob through the network and obtain the detections # and predictions

print('[INFO] computing object dectections...')
net.setInput( blob )
detections= net.forward()

print(detections.shape)

[INFO] computing object dectections...
(1, 1, 200, 7)


In [5]:
#loop over the detections 

for i in range (0, detections.shape[2]):
    confidence = detections[0,0,i,2]
    
    if confidence > 0.5:
        box= detections[0,0,i, 3:7]* np.array([w,h,w,h])
        
        (startX, startY, endX, endY)= box.astype('int')

        text= '{:.2f}%'.format(confidence * 100)
        y= startY -10 if startY - 10 > 10 else startY + 10

        # draw rectangle
        cv2.rectangle( image, (startX, startY), (endX, endY), (0,0,255), 2)
        # put text 
        cv2.putText(image, text, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0,0,255), 2)

#show the output image

cv2.imshow(" Output", image)
cv2.waitKey(0)

-1